In [1]:
import numpy as np
import random
import os
import time
import torch 
from torch import tensor
from torch.autograd import Variable
from copy import deepcopy
from random import shuffle
from tensorboardX import SummaryWriter

In [2]:
from lib.turn_program_into_file import turn_program_into_file
from game.wrappers.game_container import GameContainer

In [3]:
cpus = 4
player_size = 8
game = GameContainer(128, 128)
# Each run taks 2.6 sec per cpu, 38089 = a 24h run.
run_limit = 10
save_cycle = 2
output_dir = "./output/pyramid/"
log_path = './output/logs/'
run = int(time.time())
save_dir = output_dir + str(run) + '/'
writer = SummaryWriter()

In [4]:
# king_hill_ids = list(range(0, cpus))
os.makedirs(os.path.join(output_dir, str(run)))

In [5]:
# -*- coding: utf-8 -*-
import torch

top = 2

# Use the nn package to define our model and loss function.
p1 = torch.nn.Sequential(
    torch.nn.Linear(top**6, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.6),
    torch.nn.Linear(top**5, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.6),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


p2 = torch.nn.Sequential(
    torch.nn.Linear(top**6, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(top**5, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


loss_fn = torch.nn.BCELoss()


def return_result(game_result, number):
    result = 0
    
    for i in game_result:
        if i == number:
            result += 1
        
    return result

def create_player(game_board_pt, player_size):
    game_board_np = deepcopy(game_board_pt.detach().numpy())
    
    for i in range(len(game_board_np)):
        game_board_np[i] = 1.0 if game_board_np[i] > 0.5 else 0.0
    
    game_board_np = np.array(game_board_np, dtype=np.int)
    
    return game_board_np


def run_game(home, away, rounds, player_size):
    home = np.reshape(home, (player_size, player_size))
    away = np.reshape(away, (player_size, player_size))
    
    game.add_players(home, away)
    game.launch(rounds)
    final_board = game.gol.grid()
    final_board = np.array(final_board).flatten()
    
    return return_result(final_board, 1)

def create_loss_board(home, away, rounds, player_size, check_order):
    org_score = run_game(home, away, rounds, player_size)
    loss_board = deepcopy(home)
    tile_board = deepcopy(home)
    top_score = org_score
    
    for i in check_order:
        tile_board[i] = 0.0 if home[i] == 1.0 else 1.0
        tile_result = run_game(tile_board, away, rounds, player_size)
        if tile_result >= top_score:
            loss_board[i] = tile_board[i]
            top_score = tile_result
        else:
            tile_board = deepcopy(loss_board)
    if top_score == org_score:
        shuffle(check_order)
    
    return torch.tensor(loss_board, dtype=torch.float), org_score, top_score, check_order

learning_rate = 0.0002
check_order_p1 = list(range(0, player_size**2))
check_order_p2 = list(range(0, player_size**2))
optimizer_p1 = torch.optim.Adam(p1.parameters(), lr=learning_rate)
optimizer_p2 = torch.optim.Adam(p2.parameters(), lr=learning_rate)

start_p1 = Variable(torch.tensor(torch.rand(player_size * player_size), requires_grad=True))
start_p2 = Variable(torch.tensor(torch.rand(player_size * player_size), requires_grad=True))

for t in range(10000):
    
    p1_board = p1(start_p1)
    p2_board = p2(start_p2)

    with torch.no_grad():
        p1_board_np = create_player(p1_board.clone(), player_size)
        p2_board_np = create_player(p2_board.clone(), player_size)

        p1_loss_board, p1_score, p1_top, check_order_p1 = create_loss_board(p1_board_np, p2_board_np, 10, player_size, check_order_p1)
        p2_loss_board, p2_score, p2_top, check_order_p2 = create_loss_board(p2_board_np, p1_board_np, 10, player_size, check_order_p2)

    loss_p1 = tensor((1 - (p1_score/p1_top))) * loss_fn(p1_board, p1_loss_board)
    loss_p1.backward()

    
    optimizer_p1.step()
    
    loss_p2 = tensor((1 - (p2_score/p2_top))) * loss_fn(p2_board, p2_loss_board)
    loss_p2.backward()
    optimizer_p2.step()
    
    optimizer_p1.zero_grad()
    optimizer_p2.zero_grad()
    
    start_p1 = Variable(torch.tensor(p1_loss_board, requires_grad=True))
    start_p2 = Variable(torch.tensor(p2_loss_board, requires_grad=True))
    
    print(loss_p1, p1_score, p1_top)
    print(loss_p2, p2_score, p2_top)
    
    writer.add_scalars('drop/loss', {'loss_p1': loss_p1, 'loss_p2': loss_p2}, t)
    writer.add_scalars('drop/top', {'p1_top': p1_top, 'p2_top': p2_top}, t)
    writer.add_scalars('drop/score', {'p1_score': p1_score, 'p2_score': p2_score}, t)


    if t % 100 == 0:
        print("Saving players:")
        print(p1_board, p2_board)
#         turn_program_into_file(np.reshape(p1_board_np, (8,8)), save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
#         turn_program_into_file(np.reshape(p2_board_np, (8,8)), save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")

tensor(0.4133) 20 53
tensor(0.2796) 35 61
Saving players:
tensor([ 0.5247,  0.4535,  0.5289,  0.5523,  0.5065,  0.4807,  0.5253,
         0.5497,  0.5651,  0.5168,  0.4926,  0.5207,  0.5513,  0.4570,
         0.4620,  0.4829,  0.5153,  0.5479,  0.5759,  0.4684,  0.4996,
         0.4817,  0.4188,  0.4838,  0.5263,  0.5080,  0.4782,  0.4931,
         0.4490,  0.4634,  0.4836,  0.5217,  0.4655,  0.5017,  0.5050,
         0.4691,  0.5157,  0.4596,  0.5450,  0.4819,  0.4960,  0.4687,
         0.5764,  0.5503,  0.4788,  0.4717,  0.4998,  0.5279,  0.5026,
         0.4328,  0.4883,  0.4555,  0.4430,  0.5466,  0.5278,  0.5000,
         0.4605,  0.5874,  0.5163,  0.5044,  0.5005,  0.5116,  0.5143,
         0.5293]) tensor([ 0.5166,  0.5073,  0.5481,  0.4741,  0.4750,  0.4951,  0.4990,
         0.4735,  0.5182,  0.4663,  0.5461,  0.4895,  0.4393,  0.4594,
         0.4909,  0.5093,  0.5197,  0.5190,  0.5309,  0.4640,  0.4653,
         0.5000,  0.4690,  0.5480,  0.5054,  0.5179,  0.5249,  0.5383,
 

tensor(0.5938) 3 55
tensor(0.6489) 0 46
tensor(0.5627) 8 56
tensor(0.3562) 22 50
tensor(0.5021) 12 54
tensor(0.3808) 18 45
tensor(0.5395) 8 51
tensor(0.4770) 15 57
tensor(0.3279) 25 51
tensor(0.2574) 28 47
tensor(0.3260) 29 59
tensor(0.1664) 36 49
tensor(0.1450) 35 45
tensor(0.4307) 20 62
tensor(0.4700) 14 51
tensor(0.2845) 28 51
tensor(0.2622) 26 45
tensor(0.5383) 6 40
tensor(0.4534) 17 59
tensor(0.1075) 38 46
tensor(0.4127) 18 50
tensor(0.3055) 34 66
tensor(0.5140) 13 62
tensor(0.2456) 42 68
tensor(0.4755) 18 66
tensor(0.3486) 26 59
tensor(0.1241) 38 48
tensor(0.3972) 23 61
tensor(0.2337) 39 61
tensor(0.4625) 15 56
tensor(0.2368) 34 54
tensor(1.00000e-02 *
       9.5625) 45 53
tensor(0.4106) 21 61
tensor(0.3259) 33 68
tensor(0.2934) 29 54
tensor(0.3494) 25 54
tensor(0.3574) 28 62
tensor(0.4298) 15 48
tensor(0.3402) 30 63
tensor(0.4822) 13 54
tensor(0.4838) 17 62
tensor(0.4944) 12 58
tensor(0.5988) 3 52
tensor(0.4357) 16 52
tensor(0.4536) 18 66
tensor(0.3514) 23 52
tensor(0.2646) 37 6

tensor(0.5287) 6 66
tensor(0.1507) 30 41
tensor(0.4583) 12 64
tensor(0.2635) 28 57
tensor(0.4233) 20 61
tensor(0.4037) 11 54
tensor(0.3893) 19 55
tensor(0.3156) 25 60
tensor(0.4186) 17 53
tensor(0.3034) 21 51
tensor(0.3595) 25 59
tensor(0.2981) 23 60
tensor(1.00000e-02 *
       5.9031) 47 53
tensor(0.3156) 21 51
tensor(1.00000e-02 *
       7.5229) 42 48
tensor(0.4419) 9 63
Saving players:
tensor([ 0.5499,  0.4329,  0.4755,  0.5158,  0.5183,  0.5583,  0.5780,
         0.5057,  0.5831,  0.5631,  0.4489,  0.5752,  0.5428,  0.4240,
         0.4428,  0.3818,  0.5062,  0.5429,  0.4659,  0.4620,  0.4973,
         0.4339,  0.4719,  0.5424,  0.4119,  0.5144,  0.4374,  0.6056,
         0.4237,  0.4534,  0.4028,  0.5492,  0.4413,  0.5116,  0.4740,
         0.5232,  0.6172,  0.5012,  0.5857,  0.3949,  0.4863,  0.5274,
         0.5648,  0.5681,  0.6083,  0.5986,  0.3493,  0.5021,  0.4158,
         0.4637,  0.4262,  0.4724,  0.4281,  0.5609,  0.5413,  0.5571,
         0.4360,  0.5833,  0.6430,  0.59

tensor(0.1582) 33 57
tensor(0.2406) 22 59
tensor(0.2500) 24 48
tensor(0.1615) 30 41
tensor(0.4581) 9 80
tensor(0.2007) 21 51
tensor(0.2072) 33 54
tensor(0.1203) 33 58
tensor(0.3114) 17 53
tensor(0.2285) 23 68
tensor(0.2908) 19 57
tensor(0.4934) 0 64
tensor(1.00000e-02 *
       9.3836) 39 54
tensor(0.2628) 25 58
tensor(0.1284) 38 56
tensor(0.2198) 27 63
tensor(0.3599) 15 60
tensor(0.2110) 26 62
tensor(0.2385) 23 55
tensor(0.2301) 23 49
tensor(0.1359) 38 55
tensor(0.1624) 25 37
tensor(0.4401) 11 57
tensor(0.2253) 22 51
tensor(0.3838) 12 45
tensor(0.2119) 23 68
tensor(0.1772) 41 66
tensor(0.3222) 13 49
tensor(0.2298) 23 55
tensor(0.2178) 23 68
tensor(0.1141) 38 60
tensor(0.2740) 23 69
tensor(0.3146) 25 65
tensor(0.1946) 23 68
tensor(0.4959) 10 51
tensor(0.2578) 20 54
tensor(0.2183) 26 53
tensor(0.1443) 32 51
tensor(0.2078) 25 53
tensor(0.3461) 11 51
tensor(0.2282) 28 67
tensor(0.3816) 13 58
tensor(0.3469) 18 55
tensor(0.3876) 9 62
tensor(0.2905) 21 44
tensor(0.3771) 3 44
tensor(0.3148) 20

tensor(0.2563) 28 53
tensor(0.4479) 13 52
tensor(0.1276) 37 64
tensor(0.2408) 21 63
tensor(0.1121) 39 56
tensor(0.1458) 35 60
tensor(0.3447) 22 57
tensor(0.3043) 10 68
tensor(0.3485) 28 62
tensor(0.1891) 30 57
tensor(0.4306) 10 56
tensor(0.1201) 36 60
tensor(0.2197) 21 57
tensor(1.00000e-02 *
       8.8051) 51 76
tensor(0.1304) 26 48
tensor(0.2052) 28 64
tensor(0.1862) 29 53
tensor(0.1728) 38 73
tensor(0.1757) 38 60
tensor(0.2460) 11 57
tensor(0.1564) 33 53
tensor(0.2126) 23 57
tensor(0.2042) 19 54
tensor(0.2593) 27 58
tensor(0.1546) 39 58
tensor(0.1337) 35 65
tensor(1.00000e-02 *
       9.0041) 34 54
tensor(0.1742) 23 68
tensor(0.1353) 33 55
tensor(0.2326) 23 58
tensor(0.2436) 21 56
tensor(0.1298) 40 59
tensor(0.2286) 33 60
tensor(0.1213) 32 53
tensor(0.3720) 14 51
tensor(0.1593) 23 57
tensor(0.3310) 14 54
tensor(0.2735) 15 63
tensor(0.1626) 22 57
tensor(0.1694) 32 66
Saving players:
tensor([ 0.4301,  0.0663,  0.0894,  0.8396,  0.1619,  0.0845,  0.9560,
         0.9316,  0.8851,  0.90

...

tensor(0.1555) 38 61
tensor(1.00000e-02 *
       2.6538) 59 69
tensor(1.00000e-02 *
       2.9711) 57 67
tensor(1.00000e-02 *
       7.5255) 33 65
tensor(1.00000e-02 *
       3.0320) 57 67
tensor(0.1475) 29 57
tensor(1.00000e-02 *
       3.1105) 57 67
tensor(1.00000e-02 *
       6.2964) 33 65
tensor(1.00000e-02 *
       3.3359) 57 67
tensor(1.00000e-02 *
       4.0093) 55 77
tensor(1.00000e-02 *
       2.7531) 57 67
tensor(1.00000e-02 *
       7.8645) 33 65
tensor(1.00000e-02 *
       2.6355) 57 67
tensor(1.00000e-02 *
       2.6744) 59 69
tensor(0.1493) 38 61
tensor(1.00000e-02 *
       2.5928) 59 69
tensor(0.1342) 38 61
tensor(1.00000e-02 *
       5.2879) 33 65
tensor(0.1423) 38 61
tensor(1.00000e-02 *
       6.2987) 33 65
tensor(1.00000e-02 *
       2.8562) 57 67
tensor(1.00000e-02 *
       6.6744) 33 65
tensor(0.1713) 38 61
tensor(1.00000e-02 *
       8.3525) 33 65
tensor(0.1413) 38 61
tensor(1.00000e-02 *
       2.8896) 59 69
tensor(0.1464) 38 61
tensor(1.00000e-02 *
       6.0078

tensor(0.1243) 38 61
tensor(1.00000e-02 *
       4.6772) 33 65
tensor(0.1280) 38 61
tensor(1.00000e-02 *
       1.4272) 40 65
tensor(0.1279) 38 61
tensor(1.00000e-02 *
       2.5625) 40 65
tensor(1.00000e-02 *
       2.4691) 57 67
tensor(1.00000e-02 *
       2.1684) 40 65
tensor(0.1234) 38 61
tensor(1.00000e-02 *
       3.9347) 33 65
tensor(1.00000e-02 *
       9.5724) 45 66
tensor(1.00000e-02 *
       1.8929) 40 65
tensor(0.1361) 38 61
tensor(1.00000e-03 *
       9.2733) 65 69
tensor(0.1336) 38 61
tensor(1.00000e-02 *
       2.3001) 40 65
tensor(1.00000e-02 *
       2.7025) 57 67
tensor(1.00000e-02 *
       2.3260) 33 65
tensor(0.1321) 38 61
tensor(1.00000e-02 *
       4.5313) 33 65
tensor(0.1209) 38 61
tensor(1.00000e-02 *
       2.3564) 40 65
tensor(0.1211) 38 61
tensor(1.00000e-02 *
       5.1810) 33 65
tensor(1.00000e-02 *
       2.5507) 57 67
tensor(1.00000e-02 *
       2.0552) 40 65
tensor(1.00000e-02 *
       2.6116) 57 67
tensor(1.00000e-02 *
       2.4465) 40 65
tensor(1.0000

tensor(0.1034) 38 61
tensor(0.2351) 33 70
tensor(0.1106) 38 61
tensor(0.1175) 40 61
tensor(1.00000e-02 *
       2.0934) 57 67
tensor(0.2170) 33 70
tensor(0.1194) 38 61
tensor(0.1017) 40 61
tensor(0.1015) 38 61
tensor(0.2366) 33 70
tensor(1.00000e-02 *
       2.2450) 57 67
tensor(0.2241) 33 70
tensor(1.00000e-02 *
       3.7154) 57 67
tensor(0.1105) 40 61
tensor(1.00000e-02 *
       9.2753) 38 61
tensor(0.1281) 40 61
tensor(1.00000e-02 *
       3.2804) 57 67
tensor(0.1174) 40 61
tensor(1.00000e-02 *
       2.3648) 57 67
tensor(1.00000e-02 *
       9.7006) 40 61
tensor(1.00000e-02 *
       9.9628) 38 61
tensor(0.1019) 40 61
tensor(1.00000e-02 *
       3.6154) 52 61
tensor(0.1000) 40 61
tensor(0.1198) 36 65
tensor(0.1019) 40 61
tensor(0.1096) 38 61
tensor(0.1037) 40 61
tensor(0.1008) 38 61
tensor(0.2163) 33 70
tensor(1.00000e-02 *
       3.0304) 57 67
tensor(0.2310) 33 70
tensor(0.1252) 38 61
tensor(0.2133) 33 70
tensor(1.00000e-02 *
       2.1272) 57 67
tensor(1.00000e-02 *
       9.5176

tensor(1.00000e-02 *
       7.2264) 42 61
tensor(0.1527) 40 77
tensor(1.00000e-02 *
       9.0694) 38 61
tensor(0.1113) 33 63
tensor(1.00000e-02 *
       3.3500) 52 61
tensor(0.1115) 33 63
tensor(1.00000e-02 *
       8.2459) 38 61
tensor(0.1561) 40 77
tensor(1.00000e-02 *
       9.3435) 38 61
tensor(1.00000e-02 *
       2.0127) 59 65
tensor(1.00000e-02 *
       3.0111) 52 61
tensor(0.1027) 33 63
tensor(1.00000e-02 *
       2.0770) 57 67
tensor(0.1163) 33 63
tensor(1.00000e-02 *
       2.2165) 57 67
tensor(0.1065) 43 59
tensor(1.00000e-02 *
       2.2446) 57 67
tensor(0.1036) 33 63
tensor(1.00000e-02 *
       2.5734) 57 67
tensor(0.1457) 40 77
tensor(1.00000e-02 *
       7.4157) 38 61
tensor(1.00000e-02 *
       9.7142) 33 63
tensor(1.00000e-02 *
       9.7309) 38 61
tensor(0.1123) 43 59
tensor(0.1360) 28 67
tensor(0.1006) 33 63
tensor(1.00000e-02 *
       8.8804) 38 61
tensor(0.1455) 40 77
tensor(1.00000e-02 *
       3.3221) 52 61
tensor(0.1073) 33 63
tensor(1.00000e-02 *
       3.6504

tensor(1.00000e-02 *
       9.3201) 37 61
tensor(0.1612) 28 56
tensor(1.00000e-02 *
       1.1126) 61 67
tensor(0.1178) 43 59
tensor(1.00000e-02 *
       2.7924) 52 61
tensor(1.00000e-02 *
       7.2551) 33 63
tensor(1.00000e-02 *
       2.7382) 52 61
tensor(1.00000e-02 *
       7.4205) 33 63
tensor(1.00000e-02 *
       2.4780) 52 61
tensor(1.00000e-02 *
       7.7215) 33 63
tensor(1.00000e-02 *
       2.6696) 52 61
tensor(1.00000e-02 *
       6.7198) 33 63
tensor(1.00000e-02 *
       7.2544) 38 61
tensor(1.00000e-02 *
       8.2421) 33 63
tensor(1.00000e-02 *
       7.4635) 42 61
tensor(1.00000e-02 *
       7.5227) 33 63
tensor(1.00000e-02 *
       3.3099) 52 61
tensor(1.00000e-02 *
       8.4502) 33 63
tensor(1.00000e-02 *
       1.0711) 61 67
tensor(0.1470) 40 77
tensor(1.00000e-02 *
       7.2345) 38 61
tensor(1.00000e-02 *
       6.6102) 33 63
tensor(1.00000e-03 *
       5.7344) 51 52
tensor(1.00000e-02 *
       1.0499) 59 65
tensor(1.00000e-02 *
       7.7711) 42 61
tensor(1.0000